In [135]:
!pip install elasticsearch[async]==7.17.0 -q
!pip install elasticsearch-dsl==7.4.0 -q

In [547]:
import sys

sys.path.append('/app')

In [548]:
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd
import ast
import json

In [549]:
from memlish.config import ESTag

In [576]:
client = Elasticsearch("http://ec2-34-245-85-126.eu-west-1.compute.amazonaws.com:9200")
index = "fluentd-*"

In [577]:
# client.delete_by_query(index=[index], body={"query": {"match_all": {}}}, size=1000000)

In [578]:
# client.delete_by_query(index=[index], body={"query": {
#     "bool": {
#       "must": {
#         "match": {
#           "log": ESTag.TIMELOG
#         }
#       }
#     }
#   }}, size=1000000)

In [579]:
# time_logs = []
# time_log_search = Search(using=client, index=index).query("multi_match", query=ESTag.TIMELOG, fields=['log'])

# for hit in time_log_search.scan():
#     _hit = {}
#     _hit = ast.literal_eval(hit.to_dict()['log'])
#     time_logs.append(_hit)

In [580]:
client.info()

{'name': 'elastic',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'oyf18TppTqyj5cwfBqvswg',
 'version': {'number': '7.0.1',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'e4efcb5',
  'build_date': '2019-04-29T12:56:03.145736Z',
  'build_snapshot': False,
  'lucene_version': '8.0.0',
  'minimum_wire_compatibility_version': '6.7.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [581]:
answ_search = Search(using=client, index=index).query("multi_match", query=ESTag.INLINE_ANSWER, fields=['log'])
choice_search = Search(using=client, index=index).query("multi_match", query=ESTag.INLINE_CHOICE, fields=['log'])

In [582]:
#cand_id - is guid. Example: 3940172d-fda5-4e02-b284-6f5e2074454c 
def find_choice_for_answer(cand_id, choices):
    for i in choices:
        choice_id = i['update']['result_id'].split('_')[1]
        if choice_id == cand_id:
            return i

In [583]:
choice_logs = []
rows = []

for hit in choice_search.scan():
    _hit = {}
    _hit = ast.literal_eval(hit.to_dict()['log'])
    _hit['update'] = json.loads(_hit['update'])
    _hit["timestamp"] = hit["@timestamp"]
    choice_logs.append(_hit)

for hit in answ_search.scan():
    result_row = {}
    _hit = ast.literal_eval(hit.to_dict()['log'])
    _hit["answer_timestamp"] = hit["@timestamp"]
    if len(_hit['template_options']) > 0:
        _hit_template_uniq_id = _hit['template_options'][0]['id'].split('_')[1]
        possible_choice = find_choice_for_answer(_hit_template_uniq_id, choice_logs)
    
    update = json.loads(_hit['update'])
    
    if possible_choice is not None:
        choice_update = possible_choice['update']
        result_row["template_choice"] = choice_update["result_id"]
        result_row["choice_timestamp"] = possible_choice['timestamp']
    
    result_row["answer_timestamp"] = _hit["answer_timestamp"]
    result_row["id"] = update["id"]
    result_row["user_id"] = update["from"]["id"]
    result_row["first_name"] = update["from"]["first_name"]
    result_row["last_name"] = update["from"].get("last_name", "")
    
    if update["from"].get("username", None) is not None:
        result_row["username"] = update["from"]["username"]
    else:
        result_row["username"] = update["from"]["id"]
        
    result_row["language_code"] = update["from"]["language_code"]
    result_row["chat_type"] = update["chat_type"]
    result_row["query"] = update["query"]
    result_row["candidates"] = _hit["template_options"]
    
    rows.append(result_row)

In [584]:
logs_df = pd.DataFrame(rows)

In [585]:
logs_df.head()

,answer_timestamp,id,user_id,first_name,last_name,username,language_code,chat_type,query,candidates,template_choice,choice_timestamp
0,2022-02-17T07:10:38.000000000+00:00,1409196998664532468,328104244,Volodymyr,Prypeshnyuk,ParadiseN,ru,supergroup,halfs don't coun,[{'id': '0_473b7a22-aaa4-4683-a42e-31dba887fe2...,NaN,NaN
1,2022-02-17T07:10:39.000000000+00:00,1409196999700621644,328104244,Volodymyr,Prypeshnyuk,ParadiseN,ru,supergroup,halfs don't count,[{'id': '0_1a480179-0f83-4364-922b-0cd748bd26c...,NaN,NaN
2,2022-02-15T19:29:10.000000000+00:00,1409196998462688484,328104244,Volodymyr,Prypeshnyuk,ParadiseN,ru,supergroup,y,[{'id': '0_f7409ddc-7c24-4c2a-939c-a095bbce05e...,NaN,NaN
3,2022-02-16T09:11:45.000000000+00:00,7351830029631005566,1711731317,V,,vprypeshniuk,ru,supergroup,w,[{'id': '0_efe01464-9e06-4d8c-8e45-42d47ebcb39...,NaN,NaN
4,2022-02-17T07:10:34.000000000+00:00,1409197000501039244,328104244,Volodymyr,Prypeshnyuk,ParadiseN,ru,supergroup,halfs,[{'id': '0_37a74727-7f95-4606-ba11-2d862ec5a37...,NaN,NaN


In [586]:
banned_users = ['daniel_kovalenko', 'ParadiseN', 'vprypeshniuk', 'kilianovski', 'ostap_hembara', 'CheshireDog1865', 'dasha_hatsosh', 'NickKlimenko', 'mariupolec']

banned_mask = logs_df['username'].apply(lambda x: x not in banned_users)

with_choice_mask = logs_df['template_choice'].notna()

selection_mask = banned_mask & with_choice_mask

In [587]:
logs_df[selection_mask]['username'].value_counts()

mochi_chim_chim_chimmy        9
TheJosee                      7
Cycoric                       5
ellegado                      5
Anayilos_M                    5
sergej_havrylyuk              4
702299132                     3
Three_part_baton              3
Dasun616                      3
AmV1488                       2
Gifk0                         2
1472269408                    2
kingchoudhry                  2
An7honym                      2
lukasssss                     2
Jared_Junior                  2
1241331520                    2
T3ch31                        1
ImruYas                       1
De_Percy_Jackson              1
Super_trolly                  1
Wooooooooooooooooooooooooo    1
Anonchar                      1
Arbazkhan6001                 1
922523433                     1
MT_6501                       1
Elgor_gonita                  1
Oh_Ki                         1
Sami_ykb                      1
aj67346                       1
1911116317                    1
Player_u

In [588]:
post_date = pd.to_datetime('2022-02-18T15:30:00.000000000+00:00')

logs_sorted_dates = logs_df[selection_mask]['answer_timestamp']

logs_sorted_dates = pd.to_datetime(logs_sorted_dates)

after_post_mask = logs_sorted_dates > post_date

In [589]:
logs_df[selection_mask & after_post_mask]['username'].value_counts()

mochi_chim_chim_chimmy        8
TheJosee                      7
Cycoric                       5
Anayilos_M                    5
ellegado                      5
702299132                     3
Dasun616                      3
1472269408                    2
1241331520                    2
Jared_Junior                  2
An7honym                      2
Gifk0                         2
AmV1488                       2
kingchoudhry                  2
lukasssss                     2
ItsmehStk                     1
prince_an                     1
Arbazkhan6001                 1
Wooooooooooooooooooooooooo    1
lexashott                     1
Super_trolly                  1
supernaut911                  1
Oh_Ki                         1
ImruYas                       1
De_Percy_Jackson              1
EmdoPL                        1
nihonnnn                      1
Anonchar                      1
922523433                     1
MT_6501                       1
Elgor_gonita                  1
T3ch31  

In [590]:
logs_df[selection_mask & after_post_mask]['username'].shape

(74,)

In [591]:
logs_df[selection_mask & after_post_mask]['username'].unique().shape

(37,)

In [593]:
logs_df[logs_df['username'] == 'TheJosee']

,answer_timestamp,id,user_id,first_name,last_name,username,language_code,chat_type,query,candidates,template_choice,choice_timestamp
1639,2022-02-18T18:15:00.000000000+00:00,3155328476864434295,734657160,José D' Giovanni,,TheJosee,es,supergroup,entusia,[{'id': '0_62553c07-8cf2-4dc0-88d3-b5ad729b88d...,NaN,NaN
1640,2022-02-18T18:15:01.000000000+00:00,3155328479222060136,734657160,José D' Giovanni,,TheJosee,es,supergroup,entusiasmo,[{'id': '0_002365bb-9766-4e67-95cc-1584408af13...,NaN,NaN
1641,2022-02-18T18:15:35.000000000+00:00,3155328477572520651,734657160,José D' Giovanni,,TheJosee,es,supergroup,luis chupalo,[{'id': '0_6cd5c2c9-983a-4e25-b29f-41927a09657...,NaN,NaN
1642,2022-02-18T18:15:38.000000000+00:00,3155328476701689561,734657160,José D' Giovanni,,TheJosee,es,supergroup,luis chupalo,[{'id': '0_a67a76b6-7d8d-427b-aef9-68ca5fab123...,NaN,NaN
1643,2022-02-18T18:15:40.000000000+00:00,3155328480161722497,734657160,José D' Giovanni,,TheJosee,es,supergroup,daniel chupalo,[{'id': '0_df3b8818-7c9c-404f-95cb-a788dbec673...,NaN,NaN
1644,2022-02-18T18:15:41.000000000+00:00,3155328476702418566,734657160,José D' Giovanni,,TheJosee,es,supergroup,daniel chupalo,[{'id': '0_1a22fc17-4a41-417b-810a-3e4eb9e7459...,0_1a22fc17-4a41-417b-810a-3e4eb9e74593,2022-02-18T18:15:47.000000000+00:00
1645,2022-02-18T18:16:03.000000000+00:00,3155328477898518516,734657160,José D' Giovanni,,TheJosee,es,supergroup,hu tao,[{'id': '0_f1efa343-ca26-4ca1-8f35-9bf635e48eb...,NaN,NaN
1646,2022-02-18T18:16:04.000000000+00:00,3155328477771442269,734657160,José D' Giovanni,,TheJosee,es,supergroup,hu tao,[{'id': '0_8c8492b9-98ac-4842-9741-3ce91e796eb...,NaN,NaN
1647,2022-02-18T18:16:06.000000000+00:00,3155328480167082102,734657160,José D' Giovanni,,TheJosee,es,supergroup,ke,[{'id': '0_d801a92f-288c-493c-b561-fc83194dcf8...,NaN,NaN
1648,2022-02-18T18:16:07.000000000+00:00,3155328476068821971,734657160,José D' Giovanni,,TheJosee,es,supergroup,keching,[{'id': '0_94ce876f-96be-41ba-8386-301032c7edf...,NaN,NaN


In [438]:
logs_df[selection_mask]['username'].value_counts().shape

(18,)

In [365]:
logs_df[selection_mask]

,answer_timestamp,id,user_id,first_name,last_name,username,language_code,chat_type,query,candidates,template_choice,choice_timestamp
57,2022-02-17T12:27:24.000000000+00:00,1660455434078788046,386604907,Роман,🐼,Anvilondre,en,private,а,[{'id': '0_6a632c46-46ea-4490-aced-1c7382b9693...,NaN,NaN
59,2022-02-17T12:27:27.000000000+00:00,1660455435179586588,386604907,Роман,🐼,Anvilondre,en,private,аня шалапонькаа,[{'id': '0_8bd25000-87af-4fb0-90f5-22e1dad4697...,NaN,NaN
61,2022-02-17T12:27:36.000000000+00:00,1660455435133499468,386604907,Роман,🐼,Anvilondre,en,private,аня шалапоньк,[{'id': '0_2713aef6-1884-40f2-8b08-57be2b07963...,NaN,NaN
62,2022-02-17T12:27:37.000000000+00:00,1660455435469200539,386604907,Роман,🐼,Anvilondre,en,private,аня шалапонька,[{'id': '0_6e8b820f-7a74-4f13-9463-f88901fa106...,NaN,NaN
65,2022-02-17T12:27:45.000000000+00:00,1660455434251284423,386604907,Роман,🐼,Anvilondre,en,private,в,[{'id': '0_d9700759-6fa1-43db-aeb2-33c69224ee7...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1068,2022-02-18T15:37:24.000000000+00:00,2036082671409221046,474062438,Prince,,prince_an,en,private,hell no,[{'id': '0_bac14752-cf47-47b5-b89a-6497b9f9335...,1_bac14752-cf47-47b5-b89a-6497b9f9335b,2022-02-18T15:37:31.000000000+00:00
1079,2022-02-18T15:40:40.000000000+00:00,6028808415392279651,1403691343,Hehe,,1403691343,en,sender,fuck,[{'id': '0_8ce9f7d0-c48d-4b29-b4fb-b50b75ef295...,NaN,NaN
1080,2022-02-18T15:40:46.000000000+00:00,6028808414998953307,1403691343,Hehe,,1403691343,en,sender,f,[{'id': '0_7ac4446b-0c59-45cd-8156-fcb8ed98076...,NaN,NaN
1081,2022-02-18T15:40:48.000000000+00:00,6028808414999811553,1403691343,Hehe,,1403691343,en,sender,porn,[{'id': '0_3da1a775-700d-49e9-8c1d-ef5daedef58...,NaN,NaN


In [311]:
logs_df[selection_mask]['query']

78                                    meme
228                           жить до 160!
379                        а у меня длинее
462                                  fever
469                                 шабака
577                           це прекрасно
590         до 15 або після 20 фантастично
600    me who's watching world's end harem
612                                 скучив
623                               гачамучи
643                        Назар суперовий
759                                симсыыы
764                              I'm sorry
768                                 not me
769                                   sexy
906                                  hello
Name: query, dtype: object